In [21]:
import pandas as pd
import geopandas as gpd 
import matplotlib.pyplot as plt
import folium 
from shapely.ops import nearest_points
from shapely.geometry import LineString
from pathlib import Path

In [22]:
DATA_PATH = Path('/data/safegraph/safegraph_open_census_data')
PREPROCESSED_DATA_PATH = Path('../../../data/preprocessed/safegraph/safegraph_open_census_data')

In [23]:
# See: https://www.safegraph.com/blog/beginners-guide-to-census
    
table_ids = [
    'B01001e1',   # Total Population
    'B19013e1',   # Median Household Income
]

cbg_field_desc = pd.read_csv(DATA_PATH / 'metadata/cbg_field_descriptions.csv')
cbg_field_desc[cbg_field_desc.table_id.isin(table_ids)]

,table_id,field_full_name,field_level_1,field_level_2,field_level_3,field_level_4,field_level_5,field_level_6,field_level_7,field_level_8
4,B01001e1,SEX BY AGE: Total: Total population -- (Estimate),Sex By Age,Total,Total Population -- (Estimate),NaN,NaN,NaN,NaN,NaN
3094,B19013e1,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS ...,Median Household Income In The Past 12 Months ...,Total,Households -- (Estimate),NaN,NaN,NaN,NaN,NaN


In [ ]:
county_fips_code = '45039'  # Fairfield County, South Carolina

!mkdir -p {PREPROCESSED_DATA_PATH}
census_data_file_names = !ls {DATA_PATH}/data/ | grep [0-9] #  | cut -f 1 -d .  # eliminate .csv suffix
county_directory = PREPROCESSED_DATA_PATH / "data/county" / county_fips_code
!mkdir -p {county_directory}
for file_name in census_data_file_names:
    !touch {county_directory}/{file_name}
    !head -n 1 "{DATA_PATH}/data/{file_name}" > {county_directory}/{file_name}
    !cat "{DATA_PATH}/data/{file_name}" | grep ^{county_fips_code}.*$ >> {county_directory}/{file_name}

In [27]:
cbg_b19 = pd.read_csv(county_directory / 'cbg_b19.csv', dtype={'census_block_group': str})
cbg_b01 = pd.read_csv(county_directory / 'cbg_b01.csv', dtype={'census_block_group': str})
cbg_data = pd.merge(cbg_b01, cbg_b19, on=['census_block_group'])
cbg_data = cbg_data[['census_block_group'] + table_ids]
cbg_data.dropna().head()

,census_block_group,B01001e1,B19013e1
0,450399601001,1101,29250.0
1,450399601002,931,39931.0
2,450399602001,1015,46898.0
3,450399602002,1260,30481.0
4,450399602003,1467,48125.0


In [28]:
cbg_data[['census_block_group'] + table_ids].head()

,census_block_group,B01001e1,B19013e1
0,450399601001,1101,29250.0
1,450399601002,931,39931.0
2,450399602001,1015,46898.0
3,450399602002,1260,30481.0
4,450399602003,1467,48125.0


In [29]:
cbg_data

,census_block_group,B01001e1,B19013e1
0,450399601001,1101,29250.0
1,450399601002,931,39931.0
2,450399602001,1015,46898.0
3,450399602002,1260,30481.0
4,450399602003,1467,48125.0
5,450399603001,1046,68902.0
6,450399603002,890,NaN
7,450399603003,1482,30625.0
8,450399603004,2257,31346.0
9,450399604001,962,26250.0


In [31]:
sum(cbg_data['B01001e1'])

23025